In [0]:
%pip install nbformat databricks-sdk[openai]==0.38.0 dspy --quiet
dbutils.library.restartPython()

In [0]:
%run ../prompts/sql_prompts_with_DSPy_playground

In [0]:
%run ../prompts/ml_prompts_with_DSPy

In [0]:
%run ../prompts/optimization_prompts_with_DSPy

In [0]:
%run ../helper/GradingModule

In [0]:
module = Module(databricks.sdk.WorkspaceClient())

In [0]:
candidate_dict = {
  "X": {
    # "SQL_Q1": "/Workspace/Users/vibhor.nigam@databricks.com/interview-scripts/interview-grading/example-notebooks-sql/01-SQL-X-no-answer-q1",
    "SQL_Q2": "/Workspace/Users/vibhor.nigam@databricks.com/interview-scripts/interview-grading/example-notebooks-sql/sql_notebook_abhay",
    # "Optimization": "/Workspace/Users/vibhor.nigam@databricks.com/interview-scripts/interview-grading/example-notebooks-optimization/Optimization-X",
    # "ML": "/Workspace/Users/vibhor.nigam@databricks.com/interview-scripts/interview-grading/example-notebooks-ml/04-Machine-Learning-X"
  }
}

human_graded_dict = {
  "X": {
    # "SQL_Q1": [0,15,15,15,15,25],
    "SQL_Q2": [15,0,15,15,15,25],
    # "Optimization": [1,0,1,1,1,1,1,0],
    # "ML": [5, 30, 25, 5, 0]
  }
}

module_list = {
  "SQL": module_dict_sql,
  # "Optimization": module_dict_optimization,
  # "ML": module_dict_ml
}

llm_models = {
  # 'llm-405B': 'databricks/databricks-meta-llama-3-1-405b-instruct',
  'llm-70B': 'databricks/databricks-meta-llama-3-3-70b-instruct'
  # 'claude': "databricks/databricks-claude-3-7-sonnet"
}

In [0]:
def evaluate_scores(predictions, human_grading):
  mismatch_dict = {}

  scores_list = predictions['score'].tolist()
  code_snippet_list = predictions['code_snippet'].tolist()
  reasoning_list = predictions['reasoning'].tolist()

  for i in range(0, len(human_grading)):
    if scores_list[i]!= human_grading[i]:
      mismatch_dict[i+1] = {
        'predicted_score': scores_list[i],
        'human_score': human_grading[i],
        'code_snippet': code_snippet_list[i],
        'reasoning': reasoning_list[i]
      }
  if not mismatch_dict:
    return None
  
  return mismatch_dict

In [0]:
import time 

test_results = {}

for i in range(0,5):
  model_dict = {}
  for k,v in llm_models.items():
    start_time = time.time()
    for candidate, values in candidate_dict.items():
      answers_dict = {}
      for section, context_path in values.items():
        candidate_mismatch = {}
        lm = dspy.LM(llm_models[k], cache=False, temperature=0)
        dspy.settings.configure(lm=lm)
        module.set_module_dict(module_list["SQL"])
        results = module.get_error_and_answer_dict(context_path)
        results['answers_dict']['candidate'] = candidate
        results['answers_dict']['section'] = section
        results['answers_dict']['context'] = results['context'] 
        answers_dict[section] = results['answers_dict']
        end_time = time.time()
        execution_time = end_time - start_time
        print(f" time taken by model {k} for execution of section {section} is {execution_time} seconds")
        answers_df = pd.concat(answers_dict.values(), ignore_index=True)
        candidate_mismatch[f'{candidate}_{section}'] = evaluate_scores(answers_df, human_graded_dict[candidate][section])

    model_dict[k] = {'answers_dict': answers_dict, 'execution_time': execution_time, 'lm':lm, 'candidate_mismatch': candidate_mismatch}
    print(f" run completed {i+1} \n")
  test_results[f'{k}_round_{i+1}'] = {
    # 'model_details': model_dict,
    'mismatch': candidate_mismatch
  }

In [0]:
test_results

In [0]:
module_list['SQL']['module_2']

In [0]:
# # Define signature to be used for metric evaluation 
# class Answer(dspy.Signature):
#   score: int = dspy.OutputField(desc="The score provided by the model")
#   instruction: str = dspy.OutputField(desc="The instruction given to the llm model")
#   code_snippet: str = dspy.OutputField(desc=""" supporting code snippet provided by the model for the score """)

In [0]:
# # Read the true answers from a database 
# true_answers_df = spark.read.table("users.abhay_jalisatgi.training_set").select("score", "code_snippet", "question").collect()
# true_answer_list = [Answer(score=row.score, instruction=row.question, code_snippet=row.code_snippet) for row in true_answers_df]

In [0]:
# # Evaluate RMSE 
# def evaluate_score(example, pred, trace=None):
#   score_list = {x.instruction: x.score for x in example}
#   score_diff_list = [(x['score'] - score_list[x['question']])**2 for x in pred if x['question'] in score_list]
#   return np.sqrt(np.mean(score_diff_list))


In [0]:
# evaluate_score(true_answer_list, result)

In [0]:
# # Perform exact answer match between the code_snippets 
# def evaluate_code_snippet(example, pred, trace=None):
#   score_list = {x.instruction: x.code_snippet for x in example}
#   score_diff_list = {x['question']: (x['code_snippet'] == score_list[x['question']]) for x in pred if x['question'] in score_list}
#   return score_diff_list

In [0]:
# evaluate_code_snippet(true_answer_list, result)